In [1]:
import pandas as pd
import numpy as np
import csv
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
f=open("../venmo_input/venmo-trans.txt.old")
pay=list(csv.reader(f))

In [17]:
date=[]
sender=[]
receiver=[]    

In [18]:
for row in pay:
    time=row[0].replace("{\"created_time\": \"","")
    time=time.replace("\"","")
    
    rece=row[1].replace(" \"target\": \"","")
    rece=rece.replace("\"","")
    
    send=row[2].replace(" \"actor\": \"","")
    send=send.replace("\"}","")
    time=datetime.datetime.strptime(time,"%Y-%m-%dT%H:%M:%SZ")
    
    date.append(time)
    sender.append(send)
    receiver.append(rece)

In [19]:
d={'time':date,'sender':sender,'receiver':receiver}
df=pd.DataFrame(d)

In [20]:
df

,receiver,sender,time
0,Jamie-Korn,Jordan-Gruber,2016-04-07 03:33:19
1,Jamie-Korn,Maryann-Berry,2016-04-07 03:33:19
2,Maryann-Berry,Ying-Mo,2016-04-07 03:33:19
3,Ying-Mo,Jamie-Korn,2016-04-07 03:34:18
4,Maddie-Franklin,Maryann-Berry,2016-04-07 03:34:58
5,Ying-Mo,Maryann-Berry,2016-04-07 03:34:00
6,Rebecca-Waychunas,Natalie-Piserchio,2016-04-07 03:31:18
7,Connor-Liebman,Nick-Shirreffs,2016-04-07 03:35:02


In [21]:
df.shape

(8, 3)

In [22]:
df.dtypes

receiver            object
sender              object
time        datetime64[ns]
dtype: object

In [9]:
type(df.receiver)

pandas.core.series.Series

In [10]:
#df.sort_values('time',ascending=True)

In [23]:
import networkx as nx
G=nx.Graph()

In [25]:
G.clear()
time_lis=[]
edge_lis=[]
node_lis=[]

for i in range(df.shape[0]):
    t = df.time.iloc[i]
    ctrl=True
#    print(i)
    
    if i==0:
        t_min = t 
        t_max = t
        time_lis.append((i,t))
    
    t1=int((t-t_max).total_seconds())
    t2=int((t-t_min).total_seconds())

    if i>0:    
        if t1<-60:
            print('case 1')
            ctrl=False
            pass
    
        if t1>60:
            print('case 2')
            time_lis=[]
            time_lis.append((i,t))
    
        if (t1 <=0) & (t2>=0):
            print('case 3')
            time_lis.append((i,t))
        
        if (t1 >0) & (t1 <=60):
            print('case 4')
            time_lis.append((i,t))            
            t_max=t
            copy_time_lis=list(time_lis)
            for j in copy_time_lis:
                if int((t_max-j[1]).total_seconds())>=60:
                    time_lis.remove(j)
                    edge_lis.remove((df.receiver.iloc[j[0]],df.sender.iloc[j[0]]))
                    node_lis.remove(df.receiver.iloc[j[0]])
                    node_lis.remove(df.sender.iloc[j[0]])
                            
        if (t2 <0) & (t1 > -60):
            print('case 5')
            time_lis.append((i,t))
                
    final_t_lis=[]
    for k in time_lis:
        final_t_lis.append(k[1])        
    final_t_lis.sort()
#    print(final_t_lis) 

    t_min=final_t_lis[0]
    t_max=final_t_lis[len(time_lis)-1]
    t_inter=int((t_max-t_min).total_seconds())
    if t_inter>60:
        print('error',int((t_max-t_min).total_seconds()))
    
    if ctrl==True:
        tup=(df.receiver.iloc[i],df.sender.iloc[i])
        edge_lis.append(tup)    
        node_lis.append(df.receiver.iloc[i])
        node_lis.append(df.sender.iloc[i])
    
    G.clear()
    G.add_nodes_from(node_lis)
    G.add_edges_from(edge_lis)
    degree=list(G.degree(node_lis).values())
#    print(sorted(degree))
    a=np.median(degree)
    print("%.2f" % a)
    

1.00
case 3
1.00
case 3
1.50
case 4
2.00
case 4
1.00
case 5
1.50
case 1
1.50
case 4
1.00
